In [ ]:
import pandas as pd
import seaborn as sns
from os.path import join as pjoin
import numpy as np

In [ ]:
# df_tgt is equal to df_fb and this is okay because for decoding we need stuff 
# at the feedback anyway
#
p2 = '/home/demitau/data_Quentin/full_experiments/_export_Saxena/feedback.parquet'
df_fb = pd.read_parquet(p2)

p = '/home/demitau/data_Quentin/full_experiments/_export_Saxena/dfev_full.parquet'
dfev_full = pd.read_parquet(p)

p = '/home/demitau/data_Quentin/full_experiments/_export_Saxena/df_perframe.parquet'
dfpf_full = pd.read_parquet(p)

In [ ]:
dfpf_full['jax1'].hist()
#plt.plot()
dfpf_full['jax2'].hist()

In [ ]:
#dfpf_full.groupby(['subject','trial_index','phase'])['error'].std()

In [ ]:
#dfpf_full['nframe'] = np.arange(len(dfpf_full))

In [ ]:
# print rough info about dataset
display(df_tgt.columns)
for col in df_tgt.columns:
    cu = df_tgt[col].unique()
    cu_to_print = cu
    if len(cu) > 10:
        cu_to_print = f'len={len(cu)}'
    print(f'{col:30}', cu_to_print)

# read raw

In [ ]:
import mne

fnf = '/home/demitau/data_Quentin/full_experiments/data2/sub01_WGPOZPEE/raw_VisuoMotor_no_filter_with_ICA_broad.fif'
raw  = mne.io.read_raw_fif(fnf)

eyes_xy = ['UADC009-2104', 'UADC010-2104']
raw.set_channel_types({eyes_xy[0]:'eog',eyes_xy[1]:'eog'})

chnames_refmeg = ['BG1-1609', 'BG2-1609', 'BG3-1609', 'BP1-1609',
                  'BP2-1609', 'BP3-1609', 'BR1-1609', 'BR2-1609',
                  'BR3-1609', 'G11-1609', 'G12-1609', 'G13-1609', 
                  'G22-1609', 'G23-1609', 'P11-1609', 'P12-1609', 
                  'P22-1609', 'P23-1609', 'Q11-1609', 'Q12-1609', 'Q13-1609', 'Q22-1609', 'Q23-1609', 
                  'R11-1609', 'R12-1609', 'R13-1609', 'R22-1609', 'R23-1609']

rawmeg = raw.pick(['meg','eog'], exclude=chnames_refmeg)
chns = rawmeg.ch_names
dat,times = rawmeg[chns]
assert times[0] < 1e-6

print( times.shape, dat.shape, rawmeg.info['sfreq'] )

df = pd.DataFrame(dat.T, columns=chns)

df['time'] = times
df['nsample'] = np.arange(len(times))

df = df.set_index('nsample');#

In [ ]:
#df.set_index?

In [ ]:
path_data ='/home/demitau/data_Quentin/full_experiments/data2'

In [ ]:
import os
exp_dir = pjoin(os.path.dirname(path_data), '_export_Saxena') 
exp_fname = pjoin(exp_dir, f'MEG_data.parquet')
df.to_parquet(exp_fname)#, compression = 'brotli')

In [ ]:
#df_fb['trigger_nsample'].max() 

# Align time

In [ ]:
dt = 1/rawmeg.info['sfreq']

dfev_full = dfev_full.rename(columns={'sample':'nsample'})
dfev_full['time'] = dfev_full['nsample'] * dt

In [ ]:
# set time_since
ti = dfpf_full.query('triggers_found').trials.min()
print(ti)
time_first_valid_REST_start = dfpf_full.query('trial_index == @ti and phase == "REST_PHASE"').time.min()
print('time_first_valid_REST_start (in dfph)',time_first_valid_REST_start)
dfpf_full['time_since'] = dfpf_full['time'] - time_first_valid_REST_start

################################# 
rbeh = df_fb.query('trial_index == @ti' ).iloc[0]
nframe = rbeh['nframe']
nsample_fb = rbeh['trigger_nsample']
print('nframe', nframe, 'nsampe_fb',nsample_fb)

r = dfev_full.query('nsample <= @nsample_fb and phase == "REST_PHASE"').tail(1).iloc[0]
time_first_valid_REST_start_ev = r['time']
dfev_full['time_since'] = dfev_full['time'] - time_first_valid_REST_start_ev
print('raw time of first valid rest = ',time_first_valid_REST_start_ev)

df['time_since'] = df['time'] - time_first_valid_REST_start_ev

In [ ]:
time_inside = 0.5 # hardcoded in exp task
df_fb['time_REST']   = df_fb['time_prevprev_phase_start'] #- time_inside
df_fb['time_REST_since']   = df_fb['time_REST'] - time_first_valid_REST_start
df_fb['time_since2'] = df_fb['time'] - time_first_valid_REST_start

# note that we cannot set up 'time_since from here to df_fb' because the shift in time is not constant
#df_tgt['time_since'] = df_tgt['time_REST'] - time_first_valid_REST_start
df_fb = df_fb.reset_index().set_index('nframe')
df_fb['time_since'] = dfpf_full.set_index('nframe')['time_since']
df_fb = df_fb.reset_index()

### dbg

In [ ]:
df_fb.query('trial_index == @ti' )[['nsample'] +df_fb.columns[-5:]]

In [ ]:
if 'trigger_nsample' in df_fb.columns:
    df_fb = df_fb.rename(columns={'trigger_nsample':'nsample'})
if 'nsample' in df_fb.columns:
    df_fb = df_fb.reset_index(drop=True).set_index('nsample')

df_fb['time_meg']       = df['time']
df_fb['time_since_meg'] = df['time_since'] 

df_fb = df_fb.reset_index()

In [ ]:
print(times[nsample_fb])
cols_ = ['nframe','trial_index','target_inds','time','time_phase_start',
                                 'time_since','time_REST']
df_fb.reset_index(drop=True).query('nsample == @nsample_fb')[cols_]

In [ ]:
dfev_full.query('nsample == @nsample_fb')[['time','phase']]

In [ ]:
df.query('nsample == @nsample_fb')[['time','time_since']]

In [ ]:
dif = df_fb['time_meg'] - df_fb['time_REST']
dif.describe()

In [ ]:
dif = df_fb['time_since_meg'] - df_fb['time_since']
dif.describe()

In [ ]:
dif = df_fb['time_since_meg'] - df_fb['time_REST_since']
dif.describe()

# Interpolate continuous variables to add to df

In [ ]:
new_time = df['time_since'].values

cols = ['jax1','jax2','joyX','joyY'] + [
    'feedbackX',
    'feedbackY',
    'org_feedbackX',
    'org_feedbackY', 'traj_dist_from_center']

# 'error_distance','feedbackX','feedbackY','org_feedbackX','org_feedbackY',
# are calculated in the experiment script only in TARGET_PHASE
# joyX,joyY are real-time positions of cursor, jax1,2 are real-time angles
motor_outputs = ['jax1','jax2']
perception = ['feedbackX','feedbackY']

cols2 = ['nframe','trial_index']
for col in cols + cols2:
    #One-dimensional linear interpolation for monotonically increasing sample points.
    df[col] = np.interp(new_time, dfpf_full['time_since'], dfpf_full[col])
for col in cols2:
    df[col] = df[col].round(0).astype(int)

In [ ]:
# check alignment
qs = 'time_since <= 20 and time_since >= 0'
cols_plot = cols[:2]
df.query(qs).plot(x='time_since',y=cols_plot, figsize=(7,2))
dfpf_full.query(qs).plot(x='time_since',y=cols_plot, figsize=(7,2))

qs = 'time_since <= 620 and time_since >= 600'
cols_plot = cols[:2]
df.query(qs).plot(x='time_since',y=cols_plot, figsize=(7,2))
dfpf_full.query(qs).plot(x='time_since',y=cols_plot, figsize=(7,2))

In [ ]:
#df_fb['time_'] = pd.to_datetime(df_fb['time'])

In [ ]:
# Create the new DataFrame with the desired time step
time_new = np.arange(0, len(dt) ) * dt
df_new = pd.DataFrame({'time': time_new})

# Method to assign 'phase' values for the new DataFrame
def assign_phase(row, df_old):
    # Find the closest original time points before and after the new time
    closest_before = df_old[df_old['time'] <= row['time']].tail(1).index[0]
    closest_after = df_old[df_old['time'] >= row['time']].head(1).index[0]
    
    # If the new time point matches an original time point exactly, use its phase directly
    if closest_before == closest_after:
        return df_old.loc[closest_before, 'phase']
    
    # Otherwise, distribute phase value based on proximity to the closest original time points
    phase_before = df_old.loc[closest_before, 'phase']
    phase_after = df_old.loc[closest_after, 'phase']
    
    # For simplicity, we'll assign the phase of the closest time point
    # This approach can be refined to consider a weighted distribution if needed
    if row['time'] - df_old.loc[closest_before, 'time'] < df_old.loc[closest_after, 'time'] - row['time']:
        return phase_before
    else:
        return phase_after

# Assign 'phase' values to the new DataFrame
df_new['phase'] = df_new.apply(assign_phase, axis=1, df_old=df)

# Display the first few rows of the new DataFrame to verify
df_new.head()


# Create input columns

In [ ]:
dfev_full = dfev_full.reset_index()
df = df.reset_index()

In [ ]:
#dfev_full

In [ ]:
if 'sample' in dfev_full:
    dfev_full['nsample'] = dfev_full['sample']
    dfev_full = dfev_full.drop(columns = ['sample'])
dfev_full = dfev_full.set_index('nsample')

dfev_full['israndom'] = dfev_full['code'] % 10 >= 5

dfev_full['target_inds'] = -1
c = dfev_full['phase'] == 'TARGET_PHASE'
cs = c &  ~dfev_full['israndom']
dfev_full.loc[cs,'target_inds']  = dfev_full.loc[cs,'code'] % 10

cs = c &  dfev_full['israndom']
dfev_full.loc[cs,'target_inds']  = dfev_full.loc[cs,'code'] % 10 - 5

In [ ]:
dfev_full['cursor_ON'] = dfev_full['phase'].isin(['FEEDBACK_PHASE'])
# also if it is at home during REST. But this requires incorporating info from behav df
# well, if have to do it anyway

for tgt in range(4):
    dfev_full[f'target{tgt}_ON'] = (dfev_full['target_inds'] == tgt) | \
        ( (dfev_full['phase'] == 'FEEDBACK_PHASE')  & (dfev_full['target_inds'].shift(1) == tgt)  )
dfev_full['break_ON'] = dfev_full['phase'].isin(['BREAK_PHASE'])
dfev_full['home_ON']  = dfev_full['phase'].isin(['REST_PHASE','FEEDBACK_PHASE','TARGET_PHASE'])

In [ ]:
# import math
# def point_in_circle(circle_center, point, circle_radius):                                                                                     
#     d = math.sqrt(math.pow(point[0]-circle_center[0], 2) +                                                                                          
#                  math.pow(point[1]-circle_center[1], 2))                                                                                           
#     return d < circle_radius

# def f(row):
#     joyX,joyY = row['jax1'],row['jax2']
#     point_in_circle(home_position, (joyX, joyY), radius_cursor * 5)   
# df['cursor_ON_REST'] = df.apply(f,1)

In [ ]:
cols = [f'target{tgt}_ON' for tgt in range(4)] + ['cursor_ON', 'break_ON', 'home_ON']
for col in cols:
    df[col] = False

#df['target_visible'] = dfev_full['target_visible']
for col in cols:
    df[col] = dfev_full[col]

for col in cols:
    df[col] = df[col].ffill().fillna(False)
    #df[col] = df[col].fillna(False)

df['phase'] = dfev_full['phase']
df['phase'] = df['phase'].ffill().fillna(False)

radius_cursor, radius_target = 8,12
home_position = (400,400)

hx,hy = home_position
#df['cursor_ON_REST_cond'] = ((hx-df['org_feedbackX'])**2 + (hy-df['org_feedbackY'])**2) < radius_cursor * 5
#df['traj_dist_from_center'] = ((hx-df['org_feedbackX'])**2 + (hy-df['org_feedbackY'])**2) < radius_cursor * 5
df['cursor_ON_REST_cond'] = df['traj_dist_from_center'] < radius_cursor * 5

df['cursor_ON_0'] = df['cursor_ON'] # just for debug
df['cursor_ON'] = df['phase'].isin(['FEEDBACK_PHASE']) | \
    ( (df['phase'] == 'REST_PHASE') & df['cursor_ON_REST_cond'])  

In [ ]:
df['phase'].isin(['FEEDBACK_PHASE']).sum() / rawmeg.info['sfreq']

In [ ]:
df['cursor_ON'].sum() / rawmeg.info['sfreq'], df['cursor_ON_0'].sum() / rawmeg.info['sfreq']

In [ ]:
# print durations
for col in cols:
    d = len( df.query(col) ) / raw.info['sfreq']
    print(col,d)

In [ ]:
# plot inputs
for col in cols:
    df[col+'_int'] = df[col].astype(int)
colints = [col+'_int' for col in cols]

qs = 'time >= 5 and time < 15'
df_ = df.query(qs)
df_.plot(x='time_since', y=['cursor_ON_int'], figsize=(7,2))  #colints)
df_.plot(x='time_since', y=['target3_ON_int','target1_ON_int'], figsize=(7,2))  #colints)
df_.plot(x='time_since', y=colints, figsize=(7,2))  #colints)

I have a pandas DataFrame df with columns 'nsample','phase','target_inds' 
containing output of this experiment. Nsample is basically number of frame since the beginning of the run.
nsample is continuos stream of intergers from 0 to N.
phase is one of several strings. 
Target_inds is one of several integers: -1,0,2,3,4. Here -1 means invalid value.
    
Possible phase values are REST_PHASE then TARGET_PHASE then FEEDBACK_PHASE then ITI_PHASE and BREAK_PHASE. 
The sequence they appear in is REST_PHASE then TARGET_PHASE then FEEDBACK_PHASE then ITI_PHASE and then again REST_PHASE and so on.
Thre are also 4 rows with phase = BREAK_PHASE. 
Cursor is visible only during FEEDBACK_PHASE
Target is visible during both TARGET_PHASE and FEEDBACK_PHASE.

I want to create new columns: 'cursor_ON', 'target1_ON','target2_ON','target3_ON','target4_ON','break_ON'. 
cursor_ON should contain True only if the cursor is visible at this sample
target1_ON should contain True only if the the target is visible at this sample and the last target_inds was equal to 1
and so on

# plots

In [ ]:
df_fb.plot(x='trial_index',y=['error_deg','perturbation'])